## An introduction to machine learning with scikit-learn

### Machine learning: the problem setting

Learning problems can be categorised into:
* *Supervised learning* Data come with additional attributes that we want to predict.(the input vector has a corresponding target value). This problem can be either:
  * *Classification* :  samples belong to two or more classes and we want to learn from already labeled data how to predict the class of unlabeled data.
  * *Regression* : if the desired output consists of one or more continuous variables, then the task is called regression. E.g Stock price prediction.

* *Unsupervised learning* :  training data consists of a set of input vectors x without any corresponding target values. The goal may be to discover groups of similar examples within the data, where it is called clustering, or to determine the distribution of data within the input space, known as density estimation, or to project the data from a high-dimensional space down to two or three dimensions for the purpose of visualization

### Loading an example dataset
`scikit-learn` comes with a few standard datasets. We'll be using the `iris` dataset for classification and the `digits` dataset for regression.

In [27]:
from sklearn import datasets

In [28]:
iris = datasets.load_iris()
digits = datasets.load_digits()

A dataset is a dictionary-like object that holds all the data and some metadata about the data. This data is stored in the `.data` member, which is a `n_samples, n_features` array. In the case of supervised problem, one or more response variables are stored in the `.target` member. 
In the case of the digits dataset, `digits.data` gives access to the features that can be used to classify the digits samples:

In [29]:
digits.data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

and `digits.target` gives the number corresponding to each digit image that we are trying to learn:

In [30]:
digits.target

array([0, 1, 2, ..., 8, 9, 8])

### Learning and predicting
In the digits dataset the goal is to predict, given an image the digit it represents. We are given samples of each of the 10 possible classes (the digits zero through nine) on which we `fit` an estimator to be able to predict the classes to which unseen samples belong.

In scikit-learn, an estimator for classification is a Python object that implements the methods `fit(X, y)` and `predict(T)`.

An example of an estimator is the class `sklearn.svm.SVC`, which implements support vector classification. The estimator’s constructor takes as arguments the model’s parameters.

For now, we will consider the estimator as a black box:

In [31]:
from sklearn import svm

In [32]:
clf = svm.SVC(gamma=0.001, C=100.)

we set the value of gamma manually. To find good values for these parameters, we can use tools such as grid search and cross validation.

The `clf` (for classifier) estimator instance is first fitted to the model; that is, it must learn from the model. This is done by passing our training set to the `fit` method. For the training set, we’ll use all the images from our dataset, except for the last image, which we’ll reserve for our predicting. We select the training set with the `[:-1]` Python syntax, which produces a new array that contains all but the last item from `digits.data`:

In [33]:
clf.fit(digits.data[:-1], digits.target[:-1]) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Now you can predict new values. In this case, you’ll predict using the last image from `digits.data`. By predicting, you’ll determine the image from the training set that best matches the last image.

In [34]:
clf.predict(digits.data[-1:])

array([8])

### Model persistence
It is possible to save a model in scikit-learn by using Python’s built-in persistence model, pickle

In [35]:
import pickle

In [36]:
s = pickle.dumps(clf)

In [37]:
clf2 = pickle.loads(s)

In [38]:
clf2.predict(digits.data[-1:])

array([8])

n the specific case of scikit-learn, it may be more interesting to use joblib’s replacement for pickle (`joblib.dump` & `joblib.load`), which is more efficient on big data but it can only pickle to the disk and not to a string:

In [42]:
from joblib import dump, load

In [43]:
dump(clf, 'filename.joblib') 

['filename.joblib']

Later, you can reload the pickled model (possibly in another Python process) with:

In [44]:
clf = load('filename.joblib') 

### Conventions

scikit-learn estimators follow certain rules to make their behavior more predictive. 

#### Type casting
Unless otherwise specified, input will be cast to `float64`:

In [45]:
import numpy as np
from sklearn import random_projection

In [46]:
rng = np.random.RandomState(0)
X = rng.rand(10, 2000)
X = np.array(X, dtype='float32')
X.dtype

dtype('float32')

In [47]:
transformer = random_projection.GaussianRandomProjection()
X_new = transformer.fit_transform(X)
X_new.dtype

dtype('float64')

In this example, `X` is `float32`, which is cast to `float64` by `fit_transform(X)`.

Regression targets are cast to `float64` and classification targets are maintained:

In [48]:
from sklearn import datasets
from sklearn.svm import SVC

In [49]:
iris = datasets.load_iris()

In [53]:
clf = SVC(gamma=0.001)
clf.fit(iris.data, iris.target)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [54]:
list(clf.predict(iris.data[:3]))

[0, 0, 0]

In [55]:
clf.fit(iris.data, iris.target_names[iris.target])  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
list(clf.predict(iris.data[:3]))  

['setosa', 'setosa', 'setosa']